<a href="https://colab.research.google.com/github/SAYAN-2000/Twitter-Sentiment-Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
import re
import nltk
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
val = pd.read_csv("twitter_validation.csv", header=None)
train = pd.read_csv("twitter_training.csv", header=None)


In [8]:
train.head(10)

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
5,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
6,2402,Borderlands,Positive,So I spent a few hours making something for fu...
7,2402,Borderlands,Positive,So I spent a couple of hours doing something f...
8,2402,Borderlands,Positive,So I spent a few hours doing something for fun...
9,2402,Borderlands,Positive,So I spent a few hours making something for fu...


In [9]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       74682 non-null  int64 
 1   1       74682 non-null  object
 2   2       74682 non-null  object
 3   3       73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [10]:
len(train[1].unique())


32

In [11]:
train[2].unique()


array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [12]:
train.rename(columns={2:"labels",1:"entity",3:"tweets"}, inplace=True)


In [13]:
train["entity"].replace(to_replace="PlayerUnknownsBattlegrounds(PUBG)", value="PUBG", inplace=True)


In [14]:
train.head()

,0,entity,labels,tweets
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [15]:
targets = np.array(train["labels"])

frequency_dict = {}

for i in targets:
    if i not in frequency_dict:
        frequency_dict[i] = 1
    else:
        frequency_dict[i] +=1

labels = list(frequency_dict.keys())
frequency = list(frequency_dict.values())

In [17]:
fig=px.bar(x=labels,y=frequency)
fig.update_layout(title={
        'text': "Frequency of target",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},xaxis_title='Targets',yaxis_title='Frequency')

In [18]:
targets = np.array(train["entity"])

frequency_dict = {}

for i in targets:
    if i not in frequency_dict:
        frequency_dict[i] = 1
    else:
        frequency_dict[i] +=1
    
labels = []
frequency = list(frequency_dict.values())

for i in frequency_dict.keys():
    labels.append(i.strip())

fig=px.bar(x=labels,y=frequency)

fig.update_layout(title={
        'text': "Frequency of entities",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},xaxis_title='Entities',yaxis_title='Frequency')


In [19]:
class Find_freq:
    def __init__ (self,entity):
        self.entity = entity
        
    def calculat_freq(self,label):
        
        dic = {}

        for i in self.entity:
            if i not in dic:
                dic[i] = train[train["entity"]==i][train["labels"]==label]["labels"].count()

        return dic

In [20]:
ent = train["entity"].unique()
find_freq = Find_freq(ent)

dic_pos = find_freq.calculat_freq("Positive")
dic_neg = find_freq.calculat_freq("Negative")
dic_neut = find_freq.calculat_freq("Neutral")
dic_irre = find_freq.calculat_freq("Irrelevant")

<ipython-input-19-5f9a1718f9e5>:11: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [21]:
fig=make_subplots(rows=2,cols=2,subplot_titles=("Positive twittes about entities", "Negative twittes about entities",
                                              "  Neutral twittes about entities","Irrelavent twittes about entities"))

fig.add_trace(go.Bar(x=list(dic_pos.keys()),y=list(dic_pos.values())), row=1, col=1)
fig.add_trace(go.Bar(x=list(dic_neg.keys()),y=list(dic_neg.values())), row=1, col=2)
fig.add_trace(go.Bar(x=list(dic_neut.keys()),y=list(dic_neut.values())), row=2, col=1)
fig.add_trace(go.Bar(x=list(dic_irre.keys()),y=list(dic_irre.values())), row=2, col=2)

fig.update_layout(height=1050, width=1050,
                  title={
        'text': "Twittes about entities",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},showlegend=False)

In [22]:
train.head()

,0,entity,labels,tweets
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [25]:
y = train["labels"]
X = train.drop(columns=[0,"labels"])

In [28]:
print(len(y))
print(len(X))

# Encoding labels names using label encoder from sklearn.preprocessing
encoder = LabelEncoder()
y = encoder.fit_transform(y)
print(np.unique(y))

74682
74682
[0 1 2 3]


In [29]:
print(np.unique(encoder.inverse_transform(y)))

['Irrelevant' 'Negative' 'Neutral' 'Positive']


In [30]:
nltk.download('wordnet')
nltk.download("stopwords")
stop_words = [elements.lower() for elements in set(stopwords.words("english"))]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
X


,entity,tweets
0,Borderlands,im getting on borderlands and i will murder yo...
1,Borderlands,I am coming to the borders and I will kill you...
2,Borderlands,im getting on borderlands and i will kill you ...
3,Borderlands,im coming on borderlands and i will murder you...
4,Borderlands,im getting on borderlands 2 and i will murder ...
...,...,...
74677,Nvidia,Just realized that the Windows partition of my...
74678,Nvidia,Just realized that my Mac window partition is ...
74679,Nvidia,Just realized the windows partition of my Mac ...
74680,Nvidia,Just realized between the windows partition of...


In [32]:
X.drop(columns="entity", inplace =True)


In [33]:
X = X["tweets"].str.lower()
X.replace("[^a-zA-Z]"," ", regex=True, inplace=True)
X.head()

0    im getting on borderlands and i will murder yo...
1    i am coming to the borders and i will kill you...
2    im getting on borderlands and i will kill you ...
3    im coming on borderlands and i will murder you...
4    im getting on borderlands   and i will murder ...
Name: tweets, dtype: object

In [34]:
def remove_stop (x):
    return ",".join([words for words in str(x).split() if words not in stop_words])

X = X.apply(lambda x: remove_stop(x))
X.head()

0    im,getting,borderlands,murder
1              coming,borders,kill
2      im,getting,borderlands,kill
3     im,coming,borderlands,murder
4    im,getting,borderlands,murder
Name: tweets, dtype: object

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
    test_size=0.2,shuffle = True, random_state = 42)

In [36]:
y_train

array([2, 3, 2, ..., 1, 1, 2])

In [37]:
encoder.inverse_transform(y_train)

array(['Neutral', 'Positive', 'Neutral', ..., 'Negative', 'Negative',
       'Neutral'], dtype=object)

In [38]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(59745,)
(14937,)
(59745,)
(14937,)


In [39]:
X

0                            im,getting,borderlands,murder
1                                      coming,borders,kill
2                              im,getting,borderlands,kill
3                             im,coming,borderlands,murder
4                            im,getting,borderlands,murder
                               ...                        
74677    realized,windows,partition,mac,like,years,behi...
74678    realized,mac,window,partition,years,behind,nvi...
74679    realized,windows,partition,mac,years,behind,nv...
74680    realized,windows,partition,mac,like,years,behi...
74681    like,windows,partition,mac,like,years,behind,d...
Name: tweets, Length: 74682, dtype: object

In [40]:
x_train_array = []
for row in range(0, len(X_train.index)):
    x_train_array.append("".join(x for x in X_train.iloc[row] ))
    
    
x_test_array = []
for row in range (0, len(X_test.index)):
    x_test_array.append("".join(x for x in X_test.iloc[row]))

In [41]:
tfidfvector = TfidfVectorizer()
tfidf_train_dataset = tfidfvector.fit_transform(x_train_array)
tfidf_test_dataset = tfidfvector.transform(x_test_array)

In [42]:
encoder.inverse_transform(y_train)

array(['Neutral', 'Positive', 'Neutral', ..., 'Negative', 'Negative',
       'Neutral'], dtype=object)

In [43]:
lr_model = LR(C=10.0, max_iter=100,
                penalty='l2',
                   random_state=10, solver='liblinear' )

lr_model.fit(tfidf_train_dataset,y_train)

predictions = lr_model.predict(tfidf_test_dataset)

In [44]:
matrix=confusion_matrix(y_test,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)
kappa = cohen_kappa_score(y_test, predictions)
print(kappa)

[[2073  135  139  245]
 [  60 3991  172  296]
 [  77  200 3034  285]
 [  93  196  191 3750]]
0.8601459463078261
              precision    recall  f1-score   support

           0       0.90      0.80      0.85      2592
           1       0.88      0.88      0.88      4519
           2       0.86      0.84      0.85      3596
           3       0.82      0.89      0.85      4230

    accuracy                           0.86     14937
   macro avg       0.87      0.85      0.86     14937
weighted avg       0.86      0.86      0.86     14937

0.8104724154405449
